<hr>
<center>
   <font size='4'> <B> Tarefa: </B> Analizar questões do livro Análise de Modelos de Regressão Linear com Aplicações (CHARNET et al, 2008) </font> <br>
    <br>
    <font size='3'> <I> <B> Feito por: </B> Yasmin de Miranda Nobre </I> </font>
</center> 
<hr>

# Q6

## Do exercicio

In [ ]:
x = c(10.875,11,11.125,11.5,12,12.25,12.5,13.75,13.75,14,14.5,15.5,16.25,17,19)
y=c(71.05,70.59,69.21,68.82,69.24,68.14,67.8,67.03,67.42,66.94,68.8,65.14,64.04,63.98,62.96)

## Das analises

In [ ]:
x = c(10.875,11,11.125,11.5,12,12.25,12.5,13.75,13.75,14,14.5,15.5,16.25,17,19)
y=c(71.05,70.59,69.21,68.82,69.24,68.14,67.8,67.03,67.42,66.94,68.8,65.14,64.04,63.98,62.96)
dados = data.frame(x,y)
ajuste = lm(y~x,dados)
descritiva = function(x){
  cat("N: ", length(x), "\n")
  cat("Mínimo: ", round(min(x),4),"\n")
  cat("Média: ",  round(mean(x),4),"\n")
  cat("Mediana: ", round(median(x),4),"\n")
  cat("Máximo: ", round(max(x),4),"\n")
  cat("Desvio padrão: ", round(sd(x),4),"\n")
  cat("Coeficiente de variação: ", 100*round(sd(x)/mean(x),4),"\n")
}
#descritiva de x
stem(x,2)
descritiva(x)
#descritiva de y
stem(y,2)
descritiva(y)
#descritiva de novos valores para ip
descritiva(conf[,1])
#para fazer graficos diagnosticos
X <- model.matrix(ajuste)
y<-predict(ajuste)+resid(ajuste)
n <- nrow(X)
p <- ncol(X)
H <- X%*%solve(t(X)%*%X)%*%t(X)
h <- diag(H)
m <- fitted(ajuste)
lms <- summary(ajuste)
s <- lms$sigma
r <- resid(ajuste)
ts <- r/(s*sqrt(1-h))
di <- (1/p)*(h/(1-h))*(ts^2)
si <- lm.influence(ajuste)$sigma
tsi <- r/(si*sqrt(1-h))
#dff <- sqrt(h/(1-h))*abs(tsi) #DFFIT
ci <- sqrt( ((n-p)*h) / (p*(1-h)) )*abs(tsi) #aperfeiçoamento do DFFIT
A <- diag(r)%*%H%*%diag(r)
dmax <- abs(eigen(A)$vec[,1]/sqrt(eigen(A)$val[1]))
m <- fitted(ajuste)
#regressao modificada
dados2 = dados[-c(11,15),]
ajuste2 = lm(y~x,dados2)
summary(ajuste2)
anova(ajuste2)
t = 1/y
dados3 = data.frame(x=x,t=t)
dados3 = dados3[-11,]
ajuste3 = lm(t~x,dados3)
summary(ajuste3)
anova(ajuste3)


# Funções

In [ ]:
envel.norm <- function(modelo=fit.model,iden=0,nome=seq(along = model.matrix(modelo)[,1]),sim=100,conf=.90,res=T,quad=T) {
  
  if( class(modelo)[1]=="lm" || (class(modelo)[1]=="glm" && (modelo$family[[1]]=="Gaussian" | modelo$family[[1]]=="gaussian")) ) {
    
  } else {
    stop(paste("\nA classe do objeto deveria ser lm ou glm (com distribuicao gaussian) !!!"))
  }
  
  alfa<-(1-conf)/2
  X <- model.matrix(modelo)
  y<-predict(modelo)+resid(modelo)
  n <- nrow(X)
  p <- ncol(X)
  H <- X%*%solve(t(X)%*%X)%*%t(X)
  h <- diag(H)
  m <- fitted(modelo)
  
  #para evitar divisão por 0 ao studentizar os residuos, mas tentando manter o valor exagerado da alavanca
  h[round(h,15)==1]<-0.999999999999999
  
  si <- lm.influence(modelo)$sigma
  r <- resid(modelo)
  tsi <- r/(si*sqrt(1-h))
  sigma<-summary(modelo)$sigma
  ti <- r/(sigma*sqrt(1-h))
  di <- (1/p)*(h/(1-h))*(ti^2)
  
  e <- matrix(0,n,sim)
  e1 <- numeric(n)
  e2 <- numeric(n)
  
  for(i in 1:sim) {
    resp <- rnorm(n,m,sigma)
    fit <- lm(resp~X-1)
    ti<-resid(fit)/(summary(fit)$sigma*sqrt(1-h))
    if(res==F) {
      e[,i] <- (1/p)*(h/(1-h))*(ti^2)
    } else {
      e[,i] <- ti*sqrt( (n-p-1)/(n-p-(ti^2)) )
    }	
    e[,i] <- sort(e[,i])
  }
  
  for(i in 1:n) {
    eo <- sort(e[i,])
    e1[i] <- quantile(eo,alfa)
    e2[i] <- quantile(eo,1-alfa)
  }
  
  med <- apply(e,1,median)
  
  if(quad==T) {
    par(pty="s")
  }
  if(res==F) {
    #Segundo McCullagh e Nelder (1989, pág.407) e Paula (2003, pág.57) deve-se usar qnorm((n+1:n+.5)/(2*n+1.125))
    #Segundo Neter et alli (1996, pág.597) deve-se usar qnorm((n+1:n-.125)/(2*n+0.5))
    qq<-qnorm((n+1:n+.5)/(2*n+1.125))
    plot(qq,sort(di),xlab="Quantil Meio-Normal",ylab="Distância de Cook", ylim=range(di,e1,e2), pch=16,cex.main=2,cex.lab=1.5,cex.axis=1.5)
    nome<-nome[order(di)]
    r<-sort(di)
  } else {
    qq<-qnorm((1:n-.375)/(n+.25))
    plot(qq,sort(tsi),xlab="Quantil da Normal Padrão",ylab="Resíduo Studentizado", ylim=range(tsi,e1,e2), pch=16,main="",cex.lab=2,cex.axis=1.5)
    nome<-nome[order(tsi)]
    r<-sort(tsi)
  }
  lines(qq,e1,lty=1)
  lines(qq,e2,lty=1)
  lines(qq,med,lty=2)
  while ( (!is.numeric(iden)) || (round(iden,0) != iden) || (iden < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden<-as.numeric(out)
  }
  if(iden>0) {identify(qq,r,n=iden,labels=nome)}
  if(quad==T) {
    par(pty="m")
  }
  cat("\nBanda de ",conf*100,"% de confianca, obtida por ",sim," simulacoes.\n")
}


In [ ]:
diag.norm <- function(modelo=fit.model,iden=c(0,0,0,0,0,0),nome=seq(along = model.matrix(modelo)[,1])) {
  
    if( class(modelo)[1]=="lm" || (class(modelo)[1]=="glm" && (modelo$family[[1]]=="Gaussian" | modelo$family[[1]]=="gaussian")) ) {
    
  } else {
    stop(paste("\nA classe do objeto deveria ser lm ou glm (com distribuicao gaussian) !!!"))
  }
  
  if(length(iden)<6) {
    iden<-c(-1,-1,-1,-1,-1,-1)
  }
  
  X <- model.matrix(modelo)
  n <- nrow(X)
  p <- ncol(X)
  H <- X%*%solve(t(X)%*%X)%*%t(X)
  h <- diag(H)
  
  #para evitar divisão por 0 ao studentizar os residuos, mas tentando manter o valor exagerado da alavanca
  h[round(h,15)==1]<-0.999999999999999
  
  lms <- summary(modelo)
  s <- lms$sigma
  r <- resid(modelo)
  ts <- r/(s*sqrt(1-h))
  di <- (1/p)*(h/(1-h))*(ts^2)
  si <- lm.influence(modelo)$sigma
  tsi <- r/(si*sqrt(1-h))
  #dff <- sqrt(h/(1-h))*abs(tsi) #DFFIT
  ci <- sqrt( ((n-p)*h) / (p*(1-h)) )*abs(tsi) #aperfeiçoamento do DFFIT
  A <- diag(r)%*%H%*%diag(r)
  dmax <- abs(eigen(A)$vec[,1]/sqrt(eigen(A)$val[1]))
  m <- fitted(modelo)
  
  par(mfrow=c(2,3))
  
  plot(m,di,xlab="Valor Ajustado", ylab="Distância de Cook",main="Influência na Posição", ylim=c(0,max(di,2*mean(di))), pch=16)
  abline(2*mean(di),0,lty=2, col="red")
  while ( (!is.numeric(iden[1])) || (round(iden[1],0) != iden[1]) || (iden[1] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[1]<-as.numeric(out)
  }
  if(iden[1]>0) {identify(m,di,n=iden[1],labels=nome)}
  
  plot(m,ci,xlab="Valor Ajustado", ylab="Distância de Cook Modificada",main="Influência Posição/Escala", ylim=c(0,max(ci,2*mean(ci))), pch=16)
  abline(2*mean(ci),0,lty=2, col="red")
  while ( (!is.numeric(iden[2])) || (round(iden[2],0) != iden[2]) || (iden[2] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[2]<-as.numeric(out)
  }
  if(iden[2]>0) {identify(m,ci,n=iden[2],labels=nome)}
  
  plot(m,dmax,xlab="Valor Ajustado", ylab="Medida dmax",main="Influência Local", ylim=c(0,max(dmax,2*mean(dmax))), pch=16)
  abline(2*mean(dmax),0,lty=2, col="red")
  while ( (!is.numeric(iden[3])) || (round(iden[3],0) != iden[3]) || (iden[3] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[3]<-as.numeric(out)
  }
  if(iden[3]>0) {identify(m,dmax,n=iden[3],labels=nome)}
  
  plot(m,h,xlab="Valor Ajustado", ylab=expression(Medida~h[ii]),main="Pontos Alavanca", ylim=c(0,max(h,2*p/n)), pch=16)
  abline(2*p/n,0,lty=2, col="red")
  while ( (!is.numeric(iden[4])) || (round(iden[4],0) != iden[4]) || (iden[4] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[4]<-as.numeric(out)
  }
  if(iden[4]>0) {identify(m,h,n=iden[4],labels=nome)}
  
  plot(m,tsi,xlab="Valor Ajustado", ylab="Resíduo Studentizado",main="Pontos Aberrantes", ylim=c(-4,4), pch=16)
  abline(0,0,col="red")
  abline(3,0,lty=2)
  abline(-3,0,lty=2)
  while ( (!is.numeric(iden[5])) || (round(iden[5],0) != iden[5]) || (iden[5] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[5]<-as.numeric(out)
  }
  if(iden[5]>0) {identify(m,tsi,n=iden[5],labels=nome)}
  
  plot(m,abs(tsi),xlab="Valor Ajustado", ylab="Resíduo Padronizado Absoluto",main="Função de Variância", pch=16)
  lines(lowess(m,abs(tsi)),col="red")
  while ( (!is.numeric(iden[6])) || (round(iden[6],0) != iden[6]) || (iden[6] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[6]<-as.numeric(out)
  }
  if(iden[6]>0) {identify(m,abs(tsi),n=iden[6],labels=nome)}
  
  par(mfrow=c(1,1))
  list(ResPearsonStd=tsi,Di=di,Ci=ci,Dmax=dmax,h=h)
}

In [ ]:
#selecionando gráficos e diminuindo margem
diag.norm <- function(modelo=fit.model,iden=c(0,0,0,0,0,0),nome=seq(along = model.matrix(modelo)[,1])) {
  
  if( class(modelo)[1]=="lm" || (class(modelo)[1]=="glm" && (modelo$family[[1]]=="Gaussian" | modelo$family[[1]]=="gaussian")) ) {
    
  } else {
    stop(paste("\nA classe do objeto deveria ser lm ou glm (com distribuicao gaussian) !!!"))
  }
  
  if(length(iden)<6) {
    iden<-c(-1,-1,-1,-1,-1,-1)
  }
  
  X <- model.matrix(modelo)
  n <- nrow(X)
  p <- ncol(X)
  H <- X%*%solve(t(X)%*%X)%*%t(X)
  h <- diag(H)
  
  #para evitar divisão por 0 ao studentizar os residuos, mas tentando manter o valor exagerado da alavanca
  h[round(h,15)==1]<-0.999999999999999
  
  lms <- summary(modelo)
  s <- lms$sigma
  r <- resid(modelo)
  ts <- r/(s*sqrt(1-h))
  di <- (1/p)*(h/(1-h))*(ts^2)
  si <- lm.influence(modelo)$sigma
  tsi <- r/(si*sqrt(1-h))
  #dff <- sqrt(h/(1-h))*abs(tsi) #DFFIT
  ci <- sqrt( ((n-p)*h) / (p*(1-h)) )*abs(tsi) #aperfeiçoamento do DFFIT
  A <- diag(r)%*%H%*%diag(r)
  dmax <- abs(eigen(A)$vec[,1]/sqrt(eigen(A)$val[1]))
  m <- fitted(modelo)
  par(mfrow=c(2,2))
  
  plot(m,di,xlab="Valor Ajustado", ylab="Distância de Cook",main="Influência na Posição", ylim=c(0,max(di,2*mean(di))), pch=16)
  abline(2*mean(di),0,lty=2, col="red")
  while ( (!is.numeric(iden[1])) || (round(iden[1],0) != iden[1]) || (iden[1] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[1]<-as.numeric(out)
  }
  if(iden[1]>0) {identify(m,di,n=iden[1],labels=nome)}
  par()
  plot(m,ci,xlab="Valor Ajustado", ylab="Distância de Cook Modificada",main="Influência Posição/Escala", ylim=c(0,max(ci,2*mean(ci))), pch=16)
  abline(2*mean(ci),0,lty=2, col="red")
  while ( (!is.numeric(iden[2])) || (round(iden[2],0) != iden[2]) || (iden[2] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[2]<-as.numeric(out)
  }
  if(iden[2]>0) {identify(m,ci,n=iden[2],labels=nome)}
  
  plot(m,dmax,xlab="Valor Ajustado", ylab="Medida dmax",main="Influência Local", ylim=c(0,max(dmax,2*mean(dmax))), pch=16)
  abline(2*mean(dmax),0,lty=2, col="red")
  while ( (!is.numeric(iden[3])) || (round(iden[3],0) != iden[3]) || (iden[3] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[3]<-as.numeric(out)
  }
  if(iden[3]>0) {identify(m,dmax,n=iden[3],labels=nome)}
  
  
}

# Dos gráficos

In [ ]:
#1
par(mar=c(5, 6, 5, 4))
plot(x,y, main = , ylab="Peso (kg)", xlab = "Tempo de exercícios semanais (h)",type="p",pch=16, cex.axis=1.3,cex.lab=1.5)
#2
par(mar=c(6,6,6,6))
plot(dados$x,dados$y,main="", 
     xlab="Tempo de exercícios semanais (h)", ylab="Peso(kg)",type="p",pch=16, cex.axis=1.3, cex.lab=1.5,ylim=c(58,75),xlim=c(10,20))
conf = predict(ajuste, data.frame(x=novos), interval="prediction", level = 0.95)
abline(ajuste,lwd =2,col="#b37400")
matlines(novos, conf[,2:3],col = "#329932", lty=2,lwd=2)
legend(x=c(15.3,15.3),y=c(75.24,74),legend=c(expression(Limites~IP(y[h]~', 0.95')),expression(hat(r[1])==80.29 -0.94*x)),lty=c(2,1),lwd=c(2,2),col=c("#329932","#b37400"), box.lty=0,box.col=0, cex=1.5)
#3 e 4: residuo vs valores ajustados e funcao de variancia
diag.norm <- function(modelo=fit.model,iden=c(0,0,0,0,0,0),nome=seq(along = model.matrix(modelo)[,1])) {
  
  if( class(modelo)[1]=="lm" || (class(modelo)[1]=="glm" && (modelo$family[[1]]=="Gaussian" | modelo$family[[1]]=="gaussian")) ) {
    
  } else {
    stop(paste("\nA classe do objeto deveria ser lm ou glm (com distribuicao gaussian) !!!"))
  }
  
  if(length(iden)<6) {
    iden<-c(-1,-1,-1,-1,-1,-1)
  }
  
  X <- model.matrix(modelo)
  n <- nrow(X)
  p <- ncol(X)
  H <- X%*%solve(t(X)%*%X)%*%t(X)
  h <- diag(H)
  0
  #para evitar divisão por 0 ao studentizar os residuos, mas tentando manter o valor exagerado da alavanca
  h[round(h,15)==1]<-0.999999999999999
  
  lms <- summary(modelo)
  s <- lms$sigma
  r <- resid(modelo)
  ts <- r/(s*sqrt(1-h))
  di <- (1/p)*(h/(1-h))*(ts^2)
  si <- lm.influence(modelo)$sigma
  tsi <- r/(si*sqrt(1-h))
  #dff <- sqrt(h/(1-h))*abs(tsi) #DFFIT
  ci <- sqrt( ((n-p)*h) / (p*(1-h)) )*abs(tsi) #aperfeiçoamento do DFFIT
  A <- diag(r)%*%H%*%diag(r)
  dmax <- abs(eigen(A)$vec[,1]/sqrt(eigen(A)$val[1]))
  m <- fitted(modelo)
  par(mar=c(6,6,6,6))
  plot(m,tsi,xlab="Valor Ajustado", ylab="Resíduo Studentizado",main="", ylim=c(-4,4), pch=16,cex.axis=1.3,cex.lab=1.5, cex.main=1.8)
  abline(0,0,col="red")
  abline(3,0,lty=2)
  abline(-3,0,lty=2)
  while ( (!is.numeric(iden[5])) || (round(iden[5],0) != iden[5]) || (iden[5] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[5]<-as.numeric(out)
  }
  if(iden[5]>0) {identify(m,tsi,n=iden[5],labels=nome)}
  par(mar=c(6,6,6,6))
  plot(m,abs(tsi),xlab="Valor Ajustado", ylab="Resíduo Studentizado Absoluto",main="", pch=16,cex.axis=1.3,cex.lab=1.5, cex.main=1.8)
  lines(lowess(m,abs(tsi)),col="red")
  while ( (!is.numeric(iden[6])) || (round(iden[6],0) != iden[6]) || (iden[6] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[6]<-as.numeric(out)
  }
  if(iden[6]>0) {identify(m,abs(tsi),n=iden[6],labels=nome)}
  
  
}
diag.norm(ajuste,iden=c(1,1))
#5 envelope


#6 quatro graficos de influencia
#selecionando gráficos e diminuindo margem
diag.norm <- function(modelo=fit.model,iden=c(0,0,0,0,0,0),nome=seq(along = model.matrix(modelo)[,1])) {
  
  if( class(modelo)[1]=="lm" || (class(modelo)[1]=="glm" && (modelo$family[[1]]=="Gaussian" | modelo$family[[1]]=="gaussian")) ) {
    
  } else {
    stop(paste("\nA classe do objeto deveria ser lm ou glm (com distribuicao gaussian) !!!"))
  }
  
  if(length(iden)<6) {
    iden<-c(-1,-1,-1,-1,-1,-1)
  }
  
  X <- model.matrix(modelo)
  n <- nrow(X)
  p <- ncol(X)
  H <- X%*%solve(t(X)%*%X)%*%t(X)
  h <- diag(H)
  
  #para evitar divisão por 0 ao studentizar os residuos, mas tentando manter o valor exagerado da alavanca
  h[round(h,15)==1]<-0.999999999999999
  
  lms <- summary(modelo)
  s <- lms$sigma
  r <- resid(modelo)
  ts <- r/(s*sqrt(1-h))
  di <- (1/p)*(h/(1-h))*(ts^2)
  si <- lm.influence(modelo)$sigma
  tsi <- r/(si*sqrt(1-h))
  #dff <- sqrt(h/(1-h))*abs(tsi) #DFFIT
  ci <- sqrt( ((n-p)*h) / (p*(1-h)) )*abs(tsi) #aperfeiçoamento do DFFIT
  A <- diag(r)%*%H%*%diag(r)
  dmax <- abs(eigen(A)$vec[,1]/sqrt(eigen(A)$val[1]))
  m <- fitted(modelo)
  par(mfrow=c(2,2))
  
  plot(m,di,xlab="Valor Ajustado", ylab="Distância de Cook",main="Influência na Posição", ylim=c(0,max(di,2*mean(di))), pch=16,cex.axis=1.3,cex.lab=1.5,cex.main=1.8)
  abline(2*mean(di),0,lty=2, col="red")
  while ( (!is.numeric(iden[1])) || (round(iden[1],0) != iden[1]) || (iden[1] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[1]<-as.numeric(out)
  }
  if(iden[1]>0) {identify(m,di,n=iden[1],labels=nome)}
  par()
  plot(m,ci,xlab="Valor Ajustado", ylab="Distância de Cook Modificada",main="Influência Posição/Escala", ylim=c(0,max(ci,2*mean(ci))), pch=16,cex.axis=1.3,cex.lab=1.5,cex.main=1.8)
  abline(2*mean(ci),0,lty=2, col="red")
  while ( (!is.numeric(iden[2])) || (round(iden[2],0) != iden[2]) || (iden[2] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[2]<-as.numeric(out)
  }
  if(iden[2]>0) {identify(m,ci,n=iden[2],labels=nome)}
  
  plot(m,dmax,xlab="Valor Ajustado", ylab="Medida dmax",main="Influência Local", ylim=c(0,max(dmax,2*mean(dmax))), pch=16,cex.axis=1.3,cex.lab=1.5,cex.main=1.8)
  abline(2*mean(dmax),0,lty=2, col="red")
  while ( (!is.numeric(iden[3])) || (round(iden[3],0) != iden[3]) || (iden[3] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[3]<-as.numeric(out)
  }
  if(iden[3]>0) {identify(m,dmax,n=iden[3],labels=nome)}
  
  plot(m,h,xlab="Valor Ajustado", ylab=expression(Medida~h[ii]),main="Pontos Alavanca", ylim=c(0,max(h,2*p/n)), pch=16,cex.axis=1.3,cex.lab=1.5,cex.main=1.8)
  abline(2*p/n,0,lty=2, col="red")
  while ( (!is.numeric(iden[4])) || (round(iden[4],0) != iden[4]) || (iden[4] < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden[4]<-as.numeric(out)
  }
  if(iden[4]>0) {identify(m,h,n=iden[4],labels=nome)}
}
diag.norm(ajuste,iden=c(1,1,1,1))
#7 comparacao
par(mar=c(5, 6, 6, 4))
plot(x,y, main = , ylab="Peso (kg)", xlab = "Tempo de exercícios semanais (h)",type="p",pch=16, cex.axis=1.3,cex.lab=1.5)
abline(ajuste, lwd =2,col="#b37400")
abline(ajuste2,lty=2,lwd =2,col='#674ea7')
abline(ajuste3, lty=3,lwd=3,col="#f71aaa")
legend(x=c(17,17),y=c(71,71),legend=c(expression(hat(r[1])),expression(hat(r[2])),expression(hat(r[3]))), lty=c(1,2,3),lwd=c(2,2,3),col=c("#b37400","#674ea7",'#f71aaa'),cex=1.5,box.col=0,box.lty=0)



#  Base de dados e valores utilizados Q4

## Do exercicio

In [ ]:
x = c(3,3,4,5,6,8,8,10,10,12,12,14,14,16)
y = c(3,7,3,5,7,7,16,10,12,10,8,14,16,14)

## Das analises

In [ ]:
q1_x = quantile(x,0.25)
q2_x = quantile(x,0.5)
q3_x = quantile(x,0.75)
x_a = sort(x)
x_z = sort(x, decreasing = TRUE)
u = q2_x - x_a
v = x_z - q2_x
q1_y = quantile(y,0.25)
q2_y = quantile(y,0.5)
q3_y = quantile(y,0.75)
y_a = sort(y)
y_z = sort(y, decreasing = TRUE)
u_y = q2_y - y_a
v_y = y_z - q2_y
dados = data.frame(x,y)
ajuste <- lm(y~x, data = dados)

dados1=dados[-7,]
ajuste2 = lm(y~x,data=dados1)

# Funções

In [ ]:
envel.norm <- function(modelo=fit.model,iden=0,nome=seq(along = model.matrix(modelo)[,1]),sim=100,conf=.90,res=T,quad=T) {
  
  if( class(modelo)[1]=="lm" || (class(modelo)[1]=="glm" && (modelo$family[[1]]=="Gaussian" | modelo$family[[1]]=="gaussian")) ) {
    
  } else {
    stop(paste("\nA classe do objeto deveria ser lm ou glm (com distribuicao gaussian) !!!"))
  }
  
  alfa<-(1-conf)/2
  X <- model.matrix(modelo)
  y<-predict(modelo)+resid(modelo)
  n <- nrow(X)
  p <- ncol(X)
  H <- X%*%solve(t(X)%*%X)%*%t(X)
  h <- diag(H)
  m <- fitted(modelo)
  
  #para evitar divisão por 0 ao studentizar os residuos, mas tentando manter o valor exagerado da alavanca
  h[round(h,15)==1]<-0.999999999999999
  
  si <- lm.influence(modelo)$sigma
  r <- resid(modelo)
  tsi <- r/(si*sqrt(1-h))
  sigma<-summary(modelo)$sigma
  ti <- r/(sigma*sqrt(1-h))
  di <- (1/p)*(h/(1-h))*(ti^2)
  
  e <- matrix(0,n,sim)
  e1 <- numeric(n)
  e2 <- numeric(n)
  
  for(i in 1:sim) {
    resp <- rnorm(n,m,sigma)
    fit <- lm(resp~X-1)
    ti<-resid(fit)/(summary(fit)$sigma*sqrt(1-h))
    if(res==F) {
      e[,i] <- (1/p)*(h/(1-h))*(ti^2)
    } else {
      e[,i] <- ti*sqrt( (n-p-1)/(n-p-(ti^2)) )
    }	
    e[,i] <- sort(e[,i])
  }
  
  for(i in 1:n) {
    eo <- sort(e[i,])
    e1[i] <- quantile(eo,alfa)
    e2[i] <- quantile(eo,1-alfa)
  }
  
  med <- apply(e,1,median)
  
  if(quad==T) {
    par(pty="s")
  }
  if(res==F) {
    #Segundo McCullagh e Nelder (1989, pág.407) e Paula (2003, pág.57) deve-se usar qnorm((n+1:n+.5)/(2*n+1.125))
    #Segundo Neter et alli (1996, pág.597) deve-se usar qnorm((n+1:n-.125)/(2*n+0.5))
    qq<-qnorm((n+1:n+.5)/(2*n+1.125))
    plot(qq,sort(di),xlab="Quantil Meio-Normal",ylab="Distância de Cook", ylim=range(di,e1,e2), pch=16,cex.main=2,cex.lab=1.5,cex.axis=1.5)
    nome<-nome[order(di)]
    r<-sort(di)
  } else {
    qq<-qnorm((1:n-.375)/(n+.25))
    plot(qq,sort(tsi),xlab="Quantil da Normal Padrão",ylab="Resíduo Studentizado", ylim=range(tsi,e1,e2), pch=16,main="",cex.lab=2,cex.axis=1.5)
    nome<-nome[order(tsi)]
    r<-sort(tsi)
  }
  lines(qq,e1,lty=1)
  lines(qq,e2,lty=1)
  lines(qq,med,lty=2)
  while ( (!is.numeric(iden)) || (round(iden,0) != iden) || (iden < 0) ) {
    cat("Digite o num.de pontos a ser identificado (0=nenhum) e <enter> para continuar\n")
    out <- readline()
    iden<-as.numeric(out)
  }
  if(iden>0) {identify(qq,r,n=iden,labels=nome)}
  if(quad==T) {
    par(pty="m")
  }
  cat("\nBanda de ",conf*100,"% de confianca, obtida por ",sim," simulacoes.\n")
}


# Gráficos

In [ ]:
#1
par(mar=c(5, 6, 5, 4))
plot(u,v, xlab = expression(u[i]), ylab=expression(v[i]), main = "",type="p",pch=16,xlim=c(-8,8),ylim=c(-8,8),cex.axis=1.5,cex.lab=2)
abline(a=0,b=1,col="red")
text(4,2,expression(r[1]==~u==v),col="red",cex=1.5)
#2
par(mar=c(5, 6, 5, 4))
plot(u_y,v_y, xlab = expression(s[i]), ylab=expression(t[i]), main = "",type="p",pch=16,xlim=c(-8,8),ylim=c(-8,8),cex.axis=1.5,cex.lab=2)
abline(a=0,b=1,col="red")
text(4,2,expression(r[2]==~s==t),col="red",cex=1.5)
#3
par(mar=c(5, 6, 5, 4))
plot(x,y, main = , ylab="Quantidade de respostas", xlab = "Quantidade de linhas",type="p",pch=16, cex.axis=1.3,cex.lab=1.5)
abline(a=1,b=1,lwd=1,lty=2,cex=1.3,main="reta de 45º")
text(12,11.5,"reta de 45º",cex=1.5)
#4
par(mar=c(5, 6, 5, 4))
plot(dados$x,dados$y,main="", 
     xlab="Quantidade de linhas do anúncio", ylab="Quantidade de respostas do anúncio",type="p",pch=16, cex.axis=1.3, cex.lab=1.5)
text(13.5,11.6,expression(hat(r[3])==2.28+0.80*x),col="#b37400",cex=1.3)
novos = seq(min(x),max(x),by = 0.05)
conf = predict(ajuste, data.frame(x=novos), interval="confidence", level = 0.95)
abline(ajuste, lwd=2, col="#b37400")
matlines(novos, conf[,2:3],col = "blue", lty=2,lwd=2)
legend("bottomright",legend=expression(Limites~IC(mu[x[i]]~', 0.95')),lty=2,lwd=2,col="blue", box.lty=0,box.col=0, cex=1.5)
#5
par(mar=c(5, 6, 4, 4))
plot(fitted(ajuste),tsi,xlab="Valor Ajustado",
     ylab="Resíduo Studentizado", ylim=c(-4,4), pch=16, main="",cex.axis=1.5,cex.lab=2)
abline(0,0,col="red")
abline(3,0,lty=2)
abline(-3,0,lty=2)
identify(fitted(ajuste),tsi, n=1)
#6
par(mar=c(5, 4, 4, 4))
envel.norm(ajuste,iden=1)
#7
par(mar=c(5, 6, 4, 4))
plot(m,di,xlab="Valor Ajustado", ylab="Distância de Cook",main="", ylim=c(0,max(di,2*mean(di))), pch=16,cex.axis=1.5,cex.lab=2)
abline(2*mean(di),0,lty=2, col="red")
identify(m,di)
#8
par(mar=c(5, 6, 4, 4))
plot(m,ci,xlab="Valor Ajustado", ylab="Distância de Cook Modificada",main="", ylim=c(0,max(ci,2*mean(ci))), pch=16,cex.axis=1.5,cex.lab=2)
abline(2*mean(ci),0,lty=2, col="red")
identify(m,ci)
#9
par(mar=c(5, 6, 4, 4))
plot(m,h,xlab="Valor Ajustado", ylab=expression(~Medida~h[ii]),main="", ylim=c(0,max(h,2*p/n)), pch=16,cex.axis=1.5,cex.lab=2)
abline(2*p/n,0,lty=2, col="red")
identify(m,h)

#10
par(mar=c(5, 6, 6, 4))
plot(dados$x,dados$y,main="", 
     xlab="Quantidade de linhas do anúncio", ylab="Quantidade de respostas do anúncio",type="p",pch=16, cex.axis=1.3, cex.lab=1.5)
abline(ajuste, lwd=2,col="#b37400")
abline(ajuste2,lty=2,lwd =2,col='#674ea7')
legend(x=c(7,7),y=c(6,6),legend=c("Reta de regressão original","Reta de regressão com modificação"), lty=c(1,2),lwd=c(2,2),col=c("#b37400","#674ea7"),cex=1.5,box.col=0,box.lty=0)

